<a href="https://colab.research.google.com/github/canamac/zeka-can/blob/main/MaestroBreakoutCrypto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# Gerekli kütüphaneleri yükleyin
!pip install git+https://github.com/rongardF/tvdatafeed tradingview-screener matplotlib openpyxl

import pandas as pd
import numpy as np
from tvDatafeed import TvDatafeed, Interval
from tradingview_screener import get_all_symbols
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

# Kullanıcının borsa seçimini sağlayan fonksiyon
def get_exchange_choice():
    exchanges = {
        '1': 'BINANCE',
        '2': 'MEXC',
        '3': 'FTX',
        '4': 'BYBIT'
    }

    print("Lütfen bir borsa seçin:")
    for key, value in exchanges.items():
        print(f"{key}: {value}")

    choice = input("Seçiminizi yapın (1-4): ")
    exchange = exchanges.get(choice, 'BINANCE')
    return exchange

# Kullanıcının sembol uzantısı seçmesini sağlayan fonksiyon
def get_symbol_filter():
    print("Lütfen sembol uzantısını seçin (örneğin: USDT, BTC, ETH, BNB, BUSD): ")
    symbol_extension = input("Uzantı (Varsayılan: USDT): ")
    return symbol_extension.upper() if symbol_extension else 'USDT'

# Kullanıcıdan tek veya çift zaman dilimi seçimi yapmasını isteyen fonksiyon
def get_interval_count_choice():
    print("Tarama kaç zaman diliminde yapılsın?")
    print("1: Tek zaman dilimi")
    print("2: İki zaman dilimi")
    choice = input("Seçiminizi yapın (1 veya 2): ")
    return int(choice) if choice in ['1', '2'] else 2  # Varsayılan olarak 2 zaman dilimi

# Kullanıcının zaman dilimini seçmesini sağlayan fonksiyon
def get_interval_choice(interval_count):
    intervals = {
        '1': ('15 Dakika', Interval.in_15_minute, 50),
        '2': ('30 Dakika', Interval.in_30_minute, 40),
        '3': ('45 Dakika', Interval.in_45_minute, 35),
        '4': ('1 Saat', Interval.in_1_hour, 30),
        '5': ('2 Saat', Interval.in_2_hour, 25),
        '6': ('4 Saat', Interval.in_4_hour, 20),
        '7': ('Günlük', Interval.in_daily, 20),
        '8': ('Haftalık', Interval.in_weekly, 10),
        '9': ('Aylık', Interval.in_monthly, 5)
    }

    selected_intervals = []

    if interval_count == 1:
        print("Lütfen bir zaman dilimi seçin:")
        for key, value in intervals.items():
            print(f"{key}: {value[0]}")

        choice_1 = input("Zaman dilimini seçin (1-9): ")
        interval_1 = intervals.get(choice_1, ('4 Saat', Interval.in_4_hour, 20))
        selected_intervals.append(interval_1)

    elif interval_count == 2:
        print("Lütfen iki farklı zaman dilimi seçin:")
        for key, value in intervals.items():
            print(f"{key}: {value[0]}")

        choice_1 = input("Birinci zaman dilimini seçin (1-9): ")
        choice_2 = input("İkinci zaman dilimini seçin (1-9): ")

        interval_1 = intervals.get(choice_1, ('4 Saat', Interval.in_4_hour, 20))
        interval_2 = intervals.get(choice_2, ('Günlük', Interval.in_daily, 20))

        selected_intervals.append(interval_1)
        selected_intervals.append(interval_2)

    return selected_intervals

# Breakout stratejisini uygular
def breakout_strategy(data, period):
    df = data.copy()
    df['High_Max'] = df['High'].rolling(window=period).max()
    df['Low_Min'] = df['Low'].rolling(window=period).min()
    df['Breakout_Up'] = df['Close'] > df['High_Max'].shift(1)
    return df

tv = TvDatafeed()

# Kullanıcının borsa ve sembol uzantısı seçimini sağlar
exchange = get_exchange_choice()
symbol_extension = get_symbol_filter()

# Kullanıcının taramayı kaç zaman diliminde yapmak istediğini sorar
interval_count = get_interval_count_choice()

# Kullanıcının zaman dilim(ler)ini seçmesini sağlar
intervals = get_interval_choice(interval_count)

try:
    # Kripto sembollerini çek
    cryptos = get_all_symbols(market='crypto')

    # USDT paritelerini filtrele
    usdt_pairs = [symbol for symbol in cryptos if symbol.endswith(symbol_extension) and exchange in symbol]
    usdt_pairs = sorted(usdt_pairs)
except Exception as e:
    print(f"Semboller alınırken hata oluştu: {e}")
    usdt_pairs = []  # Hata durumunda boş bir liste atayarak devam edelim

df_breakout_up = pd.DataFrame(columns=['Sembol', 'Son Fiyat', 'Breakout Türü', 'Zaman Dilimi', 'Kırılım Zamanı'])

# Seçilen zaman dilimlerine göre analiz yapar
for interval_data in intervals:
    interval_name = interval_data[0]
    interval = interval_data[1]
    period = interval_data[2]  # Zaman dilimine göre dinamik periyot

    for symbol in usdt_pairs:  # Burada 'symbols' yerine 'usdt_pairs' kullandık
        try:
            data = tv.get_hist(symbol=symbol, exchange=exchange, interval=interval, n_bars=500)
            data.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'volume': 'Volume'}, inplace=True)
            data = data.reset_index()
            breakout_data = breakout_strategy(data, period=period)
            breakout_data['datetime'] = pd.to_datetime(breakout_data['datetime'])
            breakout_data.set_index('datetime', inplace=True)
            last_signal = breakout_data.iloc[-1]
            Breakout_Up = last_signal['Breakout_Up']
            Last_Price = last_signal['Close']
            Signal_Time = last_signal.name  # Kırılım zamanı

            if Breakout_Up:
                df_breakout_up.loc[len(df_breakout_up)] = [symbol, Last_Price, 'Breakout Yukarı', interval_name, Signal_Time]
                print(f"Tamamlandı: {symbol} - {interval_name} - Breakout Yukarı - Kırılım Zamanı: {Signal_Time}")
        except Exception as e:
            print(f"Error processing {symbol}: {e}")

# Ekranda tablo olarak gösterme
print("Breakout Yukarı Sinyalleri:")
print(df_breakout_up)


  Cloning https://github.com/rongardF/tvdatafeed to /tmp/pip-req-build-r1qy9oos
  Running command git clone --filter=blob:none --quiet https://github.com/rongardF/tvdatafeed /tmp/pip-req-build-r1qy9oos
  Resolved https://github.com/rongardF/tvdatafeed to commit e6f6aaa7de439ac6e454d9b26d2760ded8dc4923
  Preparing metadata (setup.py) ... done


Lütfen bir borsa seçin:
1: BINANCE
2: MEXC
3: FTX
4: BYBIT
Lütfen sembol uzantısını seçin (örneğin: USDT, BTC, ETH, BNB, BUSD): 
Tarama kaç zaman diliminde yapılsın?
1: Tek zaman dilimi
2: İki zaman dilimi
Lütfen bir zaman dilimi seçin:
1: 15 Dakika
2: 30 Dakika
3: 45 Dakika
4: 1 Saat
5: 2 Saat
6: 4 Saat
7: Günlük
8: Haftalık
9: Aylık
Tamamlandı: MEXC:AIHUSDT - 30 Dakika - Breakout Yukarı - Kırılım Zamanı: 2024-10-02 16:00:00
Tamamlandı: MEXC:ANALOSUSDT - 30 Dakika - Breakout Yukarı - Kırılım Zamanı: 2024-10-02 16:00:00
Tamamlandı: MEXC:AOTUSDT - 30 Dakika - Breakout Yukarı - Kırılım Zamanı: 2024-10-02 16:00:00
Tamamlandı: MEXC:AURUSDT - 30 Dakika - Breakout Yukarı - Kırılım Zamanı: 2024-10-02 16:00:00
Tamamlandı: MEXC:AXUSDT - 30 Dakika - Breakout Yukarı - Kırılım Zamanı: 2024-10-02 15:00:00
Tamamlandı: MEXC:BKNUSDT - 30 Dakika - Breakout Yukarı - Kırılım Zamanı: 2024-10-02 16:00:00
Tamamlandı: MEXC:BOOUSDT - 30 Dakika - Breakout Yukarı - Kırılım Zamanı: 2024-10-02 16:00:00
Tamamlandı

ERROR:tvDatafeed.main:Connection to remote host was lost.
ERROR:tvDatafeed.main:no data, please check the exchange and symbol


Error processing MEXC:FUNUSDT: 'NoneType' object has no attribute 'rename'
Tamamlandı: MEXC:GFALUSDT - 30 Dakika - Breakout Yukarı - Kırılım Zamanı: 2024-10-02 16:00:00
Tamamlandı: MEXC:GHXUSDT - 30 Dakika - Breakout Yukarı - Kırılım Zamanı: 2024-10-02 16:00:00
Tamamlandı: MEXC:GNFTYUSDT - 30 Dakika - Breakout Yukarı - Kırılım Zamanı: 2024-10-02 16:00:00
Tamamlandı: MEXC:GXEUSDT - 30 Dakika - Breakout Yukarı - Kırılım Zamanı: 2024-10-02 15:00:00
Tamamlandı: MEXC:IMXUSDT - 30 Dakika - Breakout Yukarı - Kırılım Zamanı: 2024-10-02 16:00:00
Tamamlandı: MEXC:KDAOUSDT - 30 Dakika - Breakout Yukarı - Kırılım Zamanı: 2024-10-02 16:00:00
Tamamlandı: MEXC:KFCUSDT - 30 Dakika - Breakout Yukarı - Kırılım Zamanı: 2024-10-02 16:00:00
Tamamlandı: MEXC:KINEUSDT - 30 Dakika - Breakout Yukarı - Kırılım Zamanı: 2024-10-02 16:00:00
Tamamlandı: MEXC:KRUUSDT - 30 Dakika - Breakout Yukarı - Kırılım Zamanı: 2024-10-02 16:00:00
Tamamlandı: MEXC:LFUSDT - 30 Dakika - Breakout Yukarı - Kırılım Zamanı: 2024-10-02 